In [1]:
%pip install numerapi joblib joblib

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from numerapi import CryptoAPI
import joblib

from utils import (
    CAPI_API_KEY,
    CAPI_API_SECRET,
    LIVE_UNIVERSE_PARQUET_FILE_PATH,
    FEATURES_DATA_FILE_PATH,
    MODEL_FILE_PATH,
    SUMBISSION_FILE_PATH
)

TRAIN_START_DATE updated to: 2020-06-01, TRAIN_END_DATE updated to: 2024-07-05


In [3]:
# Use API keys to authenticate
cApi = CryptoAPI(secret_key = CAPI_API_SECRET,  public_id = CAPI_API_KEY)

live_universe = pd.read_parquet(LIVE_UNIVERSE_PARQUET_FILE_PATH)


In [4]:
features_df = pd.read_parquet(FEATURES_DATA_FILE_PATH)
live_df = pd.read_parquet(LIVE_UNIVERSE_PARQUET_FILE_PATH)

# Load the trained model
model = joblib.load(MODEL_FILE_PATH)

In [5]:
# Get the latest date available in the historical data
latest_date = features_df['date'].max()

# Filter historical data for the latest date
latest_data = features_df[features_df['date'] == latest_date]

# Merge live data with the latest historical data
live_data = latest_data[latest_data['symbol'].isin(live_df['symbol'])]

# Select the features
feature_cols = ['symbol_encoded', 'timestamp', 'size_factor', 'liquidity_factor', 'close', 'pct_chg_30', 'close_lag_30', 'close_lag_14', 'close_lag_7', 'overall_interest_rate', 'momentum_30', 'f_g_bucket_interact_ma_1', 'f_g_bucket_interact_close_lag_3']

# Ensure live_data has the necessary features
live_features = live_data[feature_cols]

# live_features.head(10)

# Predict using the model
live_data["signal"] = model.predict(live_features)

# Ensure predictions are between 0 and 1
live_data["signal"] = live_data["signal"].rank(pct=True)

# Format and save submission
live_data[['symbol', 'signal']].to_parquet(SUMBISSION_FILE_PATH)

/var/folders/pb/5sg_h26d05sf_43qbddt42080000gn/T/ipykernel_4860/3368914659.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  live_data["signal"] = model.predict(live_features)
/var/folders/pb/5sg_h26d05sf_43qbddt42080000gn/T/ipykernel_4860/3368914659.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  live_data["signal"] = live_data["signal"].rank(pct=True)


In [6]:
# Get model ids and submit models
models = cApi.get_models()
print(f'Models available: {models}')

model_id = cApi.get_models()['ana_factormodel']

print('Submitting prediction for {model_id}')
cApi.upload_predictions(SUMBISSION_FILE_PATH, model_id = model_id)

print('Submission completed')

Models available: {'ana_factormodel': '1dbe8b79-ab14-4abc-910c-ce31f9e91662'}


2024-08-16 18:07:50,303 INFO numerapi.base_api: uploading predictions...


Submitting prediction for {model_id}
Submission completed
